### 项目介绍和视频背景分离

In [1]:
# 加载视频
import cv2
import numpy as np

cap = cv2.VideoCapture('./video.mp4')

# 循环读取视频帧
while True:
    ret,frame = cap.read()
    if ret == True:
        cv2.imshow('video',frame)
    key = cv2.waitKey(1)
    
    # 用户按esc键退出
    if key & 0xFF == 27:
        break

# 最后别忘了，释放资源
cap.release()
cv2.destroyAllWindows()



In [2]:
# 背景减除(Background Subtraction)是许多基于计算机视觉任务中的主要预处理步骤
# 如果我们有完整的静止的背景帧，那么我们可以通过帧差法来计算像素差从而获取到
# 前景对象。但是在大多数情况下，我们可能没有这样的图像，所以我们需要从我们拥有
# 的任何图像中提取背景。当运动物体有阴影时，由于阴影也在移动，情况会变得更加复杂。
# 为此引入了背景减除算法，通过这一方法我们能够从视频中分离出运动的物体前景，从而
# 达到目标检测的目的

In [3]:
# 去除背景示例

In [4]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

# 创建mog对象
mog = cv2.bgsegm.createBackgroundSubtractorMOG()

while True:
    ret,frame = cap.read()
    if ret == True:
        fgmask = mog.apply(frame)
        cv2.imshow('video',fgmask)
    key = cv2.waitKey(10)
    if key & 0xFF == 27:
        break

# 最后别忘了，释放资源
cap.release()
cv2.destroyAllWindows()

In [4]:
# 形态学识别车辆

In [17]:
#加载视频
import cv2
import numpy as np

# 创建一个从视频中捕获图像帧的cap对象
cap = cv2.VideoCapture('./video.mp4')

#创建mog对象
mog = cv2.bgsegm.createBackgroundSubtractorMOG()

#获取形态学卷积核
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))

# 定义一个标准宽度和高度来过滤掉小矩形
min_w = 90
min_h = 90

# 定义一条标准线的行高（纵坐标）
line_high = 600

# 外接矩形中心点和线接触时的偏移量,用于计数，单位是像素
offset = 9
cars = []
carno = 0

# 计算外接矩形的中心点
def center(x,y,w,h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = int(x) + x1
    cy = int(y) + y1
    return cx,cy
#循环读取视频帧
while True:
    ret,frame = cap.read()
    if ret == True:
        # 将原始帧进行灰度化，然后去噪
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        # 去噪
        blur = cv2.GaussianBlur(gray,(3,3),sigmaX=10)
        fgmask = mog.apply(blur)
        # 腐蚀
        erode = cv2.erode(fgmask,kernel)
        # 膨胀
        dilate = cv2.dilate(erode,kernel,iterations=1)
        # 闭运算 (先膨胀再腐蚀) 消除内部的小块
        close = cv2.morphologyEx(dilate,cv2.MORPH_CLOSE,kernel)
        # 查找轮廓
        contours,hirarchy = cv2.findContours(close,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        # 画出检测线
        cv2.line(frame,(10,line_high),(1200,line_high),(0,255,255),2)
        # 画出所有检测出来的轮廓
        for contour in contours:
            # 最大外接矩形，返回的左上角和宽高可能不是整数
            (x,y,w,h) = cv2.boundingRect(contour)
            # 此时可能会检测出其他的矩形，所以这里我们来作一个筛选
            # 通过一个设定一个标准的长度和高度来过滤掉小矩形
            is_valid = (w >= min_w) and (h >= min_h)
            if not is_valid:
                continue
            
            # 能进行到这里的代码检测到的都是宽度和高度符合要求的轮廓
            # 即可粗略地认为这就是车的轮廓
            # 但是我们使用函数画出外接矩形的时候都是要求整数数据
            cv2.rectangle(frame,(x,y),(x+int(w),y+int(h)),(255,255,120),2)
            
            # 这个时候我们把车抽象成一点，用来检测车是否过线
            # 这个点我们可以把它设为是外接矩形中心点
            # 我们把计算中心点的代码封装成了函数
            cpoint = center(x,y,w,h)
            # 把点追加到cars列表中
            cars.append(cpoint)
            cv2.circle(frame,cpoint,2,(0,0,255),2)
            # 判断是否过检测线，在偏移误差范围内，即可认为是正常
            for (x,y) in cars:
                if y > (line_high-offset) and y < (line_high+offset):
                    # 此时我们认为落入了有效区间
                    carno += 1
                    cars.remove((x,y))
                    print(carno)
        cv2.putText(frame,"Vehicle Count:"+str(carno),(500,50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),2)
        cv2.imshow('frame',frame)
    # 用户按ESC键退出
    key = cv2.waitKey(1)
    if key & 0xFF == 27:
        break
        
# 释放资源
cap.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223


### 判断是否是车辆

### 车辆计数逻辑

### 显示车辆计数信息